In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
!pip install dbt-core

In [ ]:
!pip install dbt-snowflake

In [ ]:
from dbt.cli.main import dbtRunner
import os

def main(session):
    # Get current session details
    current_session = session
    
    # Create profiles.yml content dynamically
    profiles_content = f"""
default:
  target: prod
  outputs:
    prod:
      type: snowflake
      account: {current_session.get_current_account()}
      user: {current_session.get_current_user()}
      role: {current_session.get_current_role()}
      warehouse: {current_session.get_current_warehouse()}
      database: {current_session.get_current_database()}
      schema: {current_session.get_current_schema()}
      threads: 4
    """
    
    # Write temporary profiles file
    temp_profiles_dir = "/tmp/dbt_profiles"
    os.makedirs(temp_profiles_dir, exist_ok=True)
    with open(f"{temp_profiles_dir}/profiles.yml", "w") as f:
        f.write(profiles_content)
    
    try:
    # Run dbt with temporary profiles
        dbt = dbtRunner()
        result = dbt.invoke([
            "run",
            "--project-dir", ".",
            "--profiles-dir", temp_profiles_dir
        ])
        # return "Success"
        return result
    except Exception as inst:
        return inst
        
    
    # return "Success" if result.success else "Failed"


In [ ]:
main(session)